In [41]:
import pandas as pd
import numpy as np
import xml.dom.minidom

In [202]:
data = pd.read_csv("all_researchers.csv",index_col=0)

In [203]:
list(data[9:10]['institution_name'])[0] is np.NaN

True

In [204]:
import re
s = "Fall 56 sfgds- 2018"
re.search(r"(\d{4})", s).group(1)

'2018'

In [42]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

import requests
import urllib3
import crawlera_proxies
import json


_SESSION = requests.session()
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = crawlera_proxies.proxies

def get_page_content(url, retry=0):
    global _SESSION

    if retry == 5:
        return None

    response = _SESSION.get(url, proxies=proxies, verify=False)

    if response.status_code == 200:
        return response.content
    else:
        print(str(response.status_code) + " Code, waiting 10s before retrying")
        time.sleep(10)
        _SESSION = requests.Session()
        return get_page_content(url, retry+1)

def search_pubmed(query):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=1000&term="
    end_url = "&field=title"

    full_url = base_url + query.replace(" ", "%20") + end_url

    content = get_page_content(full_url)

    if content is None:
        return []

    x = json.loads(content)

    try:
        return x
    except KeyError:
        return []
    
def get_pubmed_pub(pubid):
    full_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + pubid + "&retmode=xml"

    content = get_page_content(full_url)
    
    
    
    if content is None:
        return []
    

    e = xml.dom.minidom.parseString(content)
    
    
    content2 = get_page_content('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=' + pubid + '&retmode=json')
    x = json.loads(content2)
    stripped_pub = list(x['result'].values())[1]
    
    
    return e, stripped_pub['pmcrefcount']

In [38]:
x = search_pubmed("(Michael Campbell[Author])")

In [12]:
x

{'esearchresult': {'count': '9',
  'idlist': ['29169778',
   '28821539',
   '27856554',
   '26086651',
   '25713092',
   '24591505',
   '24006396',
   '22665587',
   '20516354'],
  'querytranslation': 'Andrews, Tessa[Full Author Name]',
  'retmax': '9',
  'retstart': '0',
  'translationset': [{'from': 'Tessa Andrews[Author]',
    'to': 'Andrews, Tessa[Full Author Name]'}],
  'translationstack': [{'count': '9',
    'explode': 'N',
    'field': 'Full Author Name',
    'term': 'Andrews, Tessa[Full Author Name]'},
   'GROUP']},
 'header': {'type': 'esearch', 'version': '0.3'}}

In [285]:
parse_authors([None, 1])

[None]


In [294]:
def parse_authors(authors):
    authors = [x for x in authors if len(x.getElementsByTagName("LastName"))==1]
    author_names = [x.getElementsByTagName("ForeName")[0].firstChild.data + " " + x.getElementsByTagName("LastName")[0].firstChild.data for x in authors]
    return author_names

In [19]:
pub = get_pubmed_pub('27856554')[0]

In [20]:
year = pub.getElementsByTagName("MedlineDate")[0].toxml()

In [32]:
year

'<MedlineDate>Winter 2016</MedlineDate>'

In [37]:
import re
re.search('\d{4}', '').group(0)


AttributeError: 'NoneType' object has no attribute 'group'

In [39]:
for pub_id in x['esearchresult']['idlist']:
    pub = get_pubmed_pub(pub_id)[0]
    try:
        title = pub.getElementsByTagName("ArticleTitle")[0].firstChild.data
    except:
        print(pub_id)

25642549


In [ ]:
raw = pub.toxml().replace('\n', '')

In [ ]:
test = xml.dom.minidom.parseString(raw)

In [ ]:
test.getElementsByTagName("PubDate")[0].getElementsByTagName("Year")[0].firstChild.data

In [1]:
import scopus
from scopus import AuthorSearch

In [3]:
scopus.load_api_key()

In [3]:
x = scopus.AuthorSearch('AUTHLASTNAME(Abouheif) and AUTHFIRST(Ehab)')

In [4]:
import requests 
y = scopus.ScopusAuthor(x.authors[0].eid).get_abstracts()


In [11]:
import json
pre_raw = vars(y[0])
pre_raw.pop('xml', None)


In [12]:
vars(pre_raw)

TypeError: vars() argument must have __dict__ attribute